In [ ]:
# Due to the file system setting of the Jupyter Notebook, navigate to the package to use infra
%cd /home/jupyter/quant

### Batch Extract & Load Stock Price Data into GCP Cloud Storage and mySQL

In [ ]:
from datetime import datetime
import pandas as pd
import pymysql
from yitian.datasource import *

# Set up cloud sql connections
password = ''
connection = pymysql.connect(host=PRIVATE_HOST,
                             user=USER,
                             password=password,
                             db=DATABASE)

In [ ]:
# Set up default and optional parameters
nasdaq_pd = pd.read_sql("SELECT * FROM quant.nasdaq", connection)
nasdaq_tickers = nasdaq_pd.symbol

password = ''
period = 'max'
interval = '1d'
date_col = 'Date'
output_file = 'daily.csv'

instance = INSTANCE
database = DATABASE
table = STOCK_DAILY

# Check the consistency of variables between daily and hourly extractions
if interval == '60m':
    if ((period in ['5y', '10y', 'max']) | (date_col != 'Datetime') | (output_file != 'hourly.csv') | (table != STOCK_HOURLY)):
    raise ValueError(f"When Interval is '60m', period, date_col, output_file and table have to be set accordingly")

In [ ]:
# Extract data to GCP cloud storage and load to mySQL
start = datetime.now()
for t in nasdaq_tickers:
    ticker = t
    %run -i 'notebooks/etl/yfinance/stock_price_etl.py'
    
    print("{ticker} ETL is finished in {time}".format(ticker=ticker, time=start - datetime.now()))

In [ ]:
# Close connection
connection.close()